# 1/ LLM Chatbot RAGのデータ準備

## Databricks Vector Searchを用いたナレッジベースの構築とインデックス化

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/llm-rag-managed-flow-1.png?raw=true" style="float: right; width: 800px; margin-left: 10px">

このノートブックでは、ドキュメントページを取り込み、Vector Searchインデックスを作成して、チャットボットの回答精度を向上させる方法を学びます。

高品質なデータを準備することは、チャットボットの性能向上において非常に重要です。ぜひ、ご自身のデータセットを使用して、次のステップを実装してみてください。

幸いにも、Lakehouse AIは、AIおよびLLMプロジェクトを加速する最先端のソリューションを提供しており、大規模なデータ取り込みや準備を簡素化します。

この例では、[docs.databricks.com](https://docs.databricks.com) からDatabricksのドキュメントを使用します：
- ウェブページをダウンロードする
- ページを小さなテキストチャンクに分割する
- Databricks Foundationモデルを使用して埋め込みを計算し、それをDelta Tableの一部として保存する
- Delta Tableに基づいてVector Searchインデックスを作成する

<!-- 使用データの収集（ビュー）。これを削除すると、収集が無効になります。インストール中にトラッカーを無効化することも可能です。詳細はREADMEをご覧ください。 -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=2952210246698934&notebook=%2F02-simple-app%2F01-Data-Preparation-and-Index&demo_name=llm-rag-chatbot&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fllm-rag-chatbot%2F02-simple-app%2F01-Data-Preparation-and-Index&version=1">

In [0]:
%pip install --quiet mlflow lxml==4.9.3 transformers langchain databricks-vectorsearch
dbutils.library.restartPython()

In [0]:
%run ./00_config

In [0]:
# 本ノートブックで利用するスキーマを作成
schema_name = f"05_vector_search_index_for_{user_name}"
print(f"schema_name: `{schema_name}`")
spark.sql(
    f"""
    CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}
    """
)

spark.sql(f"USE {catalog_name}.{schema_name}")

In [0]:
%run ./_resources/00-init $reset_all_data=false

In [0]:
%sql
select current_schema();

## Databricksドキュメントのサイトマップとページの抽出

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/llm-rag-data-prep-1.png?raw=true" style="float: right; width: 600px; margin-left: 10px">


**※ openhackのコンテンツ用に説明を書き換える。**  
**※ openhackでは、製品のマニュアル(html)を取り込み予定。**

まず、Delta Lakeテーブルとして生データセットを作成します。

このデモでは、`docs.databricks.com` からいくつかのドキュメントページを直接ダウンロードし、HTMLコンテンツを保存します。

主な手順は以下の通りです：

- `sitemap.xml` ファイルからページのURLを抽出する簡単なスクリプトを実行
- ウェブページをダウンロード
- BeautifulSoupを使用してArticleBodyを抽出
- HTML結果をDelta Lakeテーブルに保存


In [0]:
import pyspark.sql.functions as F

# def load_html_files_to_table(input_path: str, table_name: str, mode: str = 'append'):
def load_html_files_to_table():
    input_path = "/Volumes/trainer_catalog/default/src_data/openhack-llm/*.html"
    table_name = "raw_documentation"
    mode = "append"
    # HTMLファイルを読み込み (wholetext)
    df = (
        spark.read.format("text")
        .option("wholetext", True)
        .load(input_path)
        .withColumn("filename", F.col("_metadata.file_path"))
        .withColumn("filename", F.regexp_replace(F.col("filename"), r'^.*/', ''))
        .withColumnRenamed("filename", "url")
        .withColumnRenamed("value", "text")
        .select("url", "text")
    )

    # Deltaテーブルとして書き込み
    df.write.format("delta").mode(mode).saveAsTable(table_name)

    return df

In [0]:
if not spark.catalog.tableExists("raw_documentation") or spark.table("raw_documentation").isEmpty():
    doc_articles = load_html_files_to_table()
    #Save them as a raw_documentation table
    doc_articles.write.mode('overwrite').saveAsTable("raw_documentation")

display(spark.table("raw_documentation"))


### ドキュメントページを小さなチャンクに分割する

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/llm-rag-data-prep-2.png?raw=true" style="float: right; width: 600px; margin-left: 10px">

LLMモデルには通常、最大入力コンテキスト長があり、非常に長いテキストでは埋め込みを計算できません。  
さらに、コンテキストが長くなるほど、モデルが応答を生成するのに時間がかかります。

ドキュメントの適切な準備はモデルの性能向上において重要であり、データセットに応じて以下のような戦略があります：

- ドキュメントを小さなチャンク（段落、h2など）に分割する
- ドキュメントを固定された長さに切り詰める（トランケーション）
- チャンクサイズは、コンテンツの特性やプロンプトでの使用方法によって変わる
  - 複数の小さなチャンクをプロンプトに含める場合と、大きなチャンクを1つ送信する場合では結果が異なることがある
- 大きなチャンクに分割し、それぞれをモデルで一度に要約しておくことで、リアルタイムの推論を高速化
- 複数のエージェントを用いて、大きなドキュメントを並行して評価し、最終的なエージェントが回答を生成する戦略を採用

これらの方法を試し、データセットやユースケースに合った分割方法を選択してください。


### ドキュメントページを小さなチャンク（h2セクション）に分割

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/chunk-window-size.png?raw=true" style="float: right" width="700px">
<br/>
このデモでは、長すぎてモデルへのプロンプトに収められない大きなドキュメント記事を扱います。

複数のドキュメントをRAG（Retrieval-Augmented Generation）コンテキストとして使用することは、最大入力サイズを超えてしまうため不可能です。また、最近の研究では、ウィンドウサイズを大きくすることが必ずしも効果的ではないとされており、LLM（大規模言語モデル）はプロンプトの冒頭と末尾に重点を置く傾向があると示唆されています。

ここでは、記事をHTMLの`h2`タグごとに分割し、HTMLを削除したうえで、LangChainを使用して各チャンクを500トークン未満に収めます。

#### LLMのウィンドウサイズとトークナイザー

同じ文章でも、異なるモデルでは異なるトークン数になることがあります。LLMには、与えられた文章のトークン数を数えるための`Tokenizer`が付属しており、通常、単語数よりも多くのトークンが生成されます（詳細は[Hugging Faceのドキュメント](https://huggingface.co/docs/transformers/main/tokenizer_summary)や[OpenAIのリポジトリ](https://github.com/openai/tiktoken)をご覧ください）。

ここで使用するトークナイザーがモデルと一致していることを確認してください。DatabricksのDBRX InstructモデルはGPT-4と同じトークナイザーを使用します。この例では、`transformers`ライブラリを使用して、DBRX Instructのトークナイザーでトークン数をカウントします。また、ドキュメントのトークンサイズが埋め込みの最大サイズ（1024トークン）を超えないようにします。

<br/>
<br style="clear: both">
<div style="background-color: #def2ff; padding: 15px;  border-radius: 30px;">
  <strong>情報</strong><br/>
  以下の手順はデータセットごとに異なります。これは、成功するRAGアシスタントを構築するための重要な部分です。
  <br/>作成されたチャンクを手動で確認し、内容が適切で関連性があることを必ず確認してください。
</div>


In [0]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from bs4 import BeautifulSoup
from transformers import OpenAIGPTTokenizer

max_chunk_size = 500
tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=max_chunk_size, chunk_overlap=50)

def split_html_with_h3_p(html, min_chunk_size=20, max_chunk_size=500):
    if not html:
        return []

    soup = BeautifulSoup(html, "html.parser")
    chunks = []
    current_h2 = None
    current_h3 = None
    current_chunk = ""

    for tag in soup.find_all(["h2", "h3", "p"]):
        text = tag.get_text(strip=True)
        if not text:
            continue

        if tag.name == "h2":
            # h2が新しく来たら、前のチャンクを確定
            if current_chunk:
                chunks.extend(text_splitter.split_text(current_chunk.strip()))
            current_h2 = text
            current_h3 = None  # h3をリセット
            current_chunk = f"{current_h2}\n"

        elif tag.name == "h3":
            # h3が新しく来たら、前のチャンクを確定
            if current_chunk:
                chunks.extend(text_splitter.split_text(current_chunk.strip()))
            current_h3 = text
            current_chunk = f"{current_h2}\n{current_h3}\n" if current_h2 else f"{current_h3}\n"

        elif tag.name == "p":
            # pは現在のh2またはh3に追加
            if len(tokenizer.encode(current_chunk + text)) <= max_chunk_size:
                current_chunk += text + "\n"
            else:
                chunks.extend(text_splitter.split_text(current_chunk.strip()))
                current_chunk = text + "\n"

    if current_chunk:
        chunks.extend(text_splitter.split_text(current_chunk.strip()))

    return [c for c in chunks if len(tokenizer.encode(c)) > min_chunk_size]

# DatabricksのテーブルからHTMLデータを取得
html = spark.table("raw_documentation").limit(1).collect()[0]['text']
split_chunks = split_html_with_h3_p(html)
print(split_chunks)

### チャンクの作成とDeltaテーブルへの保存

最後のステップは、UDF（ユーザー定義関数）をすべてのドキュメントテキストに適用し、それらを`databricks_documentation`テーブルに保存することです。

*この部分は通常、本番環境レベルのジョブとして設定され、新しいドキュメントページが更新され次第、実行される形になります。<br/>Delta Live Tableパイプラインとして設定し、更新を逐次的に処理することも可能です。*


In [0]:
%sql
--Note that we need to enable Change Data Feed on the table to create the index
CREATE TABLE IF NOT EXISTS product_documentation (
  id BIGINT GENERATED BY DEFAULT AS IDENTITY,
  url STRING,
  content STRING
) TBLPROPERTIES (delta.enableChangeDataFeed = true); 

In [0]:
# Let's create a user-defined function (UDF) to chunk all our documents with spark
import pandas as pd

@F.pandas_udf("array<string>")
def parse_and_split(docs: pd.Series) -> pd.Series:
    return docs.apply(split_html_with_h3_p)
    # return docs.apply(split_html_on_h2)
    
(spark.table("raw_documentation")
      .filter('text is not null')
      .withColumn('content', F.explode(parse_and_split('text')))
      .drop("text")
      .write.mode('overwrite').saveAsTable("product_documentation"))

display(spark.table("product_documentation"))

## ベクター検索インデックスの要件

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/databricks-vector-search-managed-type.png?raw=true" style="float: right" width="800px">

Databricksでは、以下の複数の種類のベクター検索インデックスを提供しています：

- **マネージド埋め込み (Managed embeddings)**  
  テキストカラムとエンドポイント名を指定すると、DatabricksがDeltaテーブルとインデックスを同期します。  
  **（このデモではこれを使用します）**

- **セルフマネージド埋め込み (Self Managed embeddings)**  
  埋め込みを自分で計算し、それをDeltaテーブルのフィールドとして保存します。その後、Databricksがインデックスを同期します。

- **直接インデックス (Direct index)**  
  Deltaテーブルを使用せずにインデックスを利用・更新したい場合に使用します。

このデモでは、**マネージド埋め込み**インデックスの設定方法を紹介します。  
（セルフマネージド埋め込みは上級デモでカバーされています。）


## Databricks GTE Embeddings Foundation Model エンドポイントの紹介

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/llm-rag-data-prep-4.png?raw=true" style="float: right; width: 600px; margin-left: 10px">

Databricksが提供するファウンデーションモデルは、すぐに利用できる形で提供されています。

Databricksでは、埋め込みを計算したりモデルを評価するために、以下のエンドポイントタイプをサポートしています：
- **ファウンデーションモデルエンドポイント**（例: DBRX, MPT, GTE）  
  Databricksが提供する標準エンドポイント。  
  **このデモではこれを使用します。**
- **外部エンドポイント**  
  外部モデルへのゲートウェイとして機能（例: Azure OpenAI）
- **カスタムモデル**  
  Databricksモデルサービスにホストされたファインチューニング済みモデル

[Model Serving Endpointページ](/ml/endpoints)を開いて、ファウンデーションモデルを探索し試してみてください。

このデモでは、埋め込み用にファウンデーションモデルの`GTE`を、チャット用に`DBRX`を使用します。<br/><br/>

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/databricks-foundation-models.png?raw=true" width="600px" >


In [0]:
import mlflow.deployments

# エンベディングモデルをtext-embedding-ada-002に変更

# Databricks のデプロイメントクライアントを取得
deploy_client = mlflow.deployments.get_deploy_client("databricks")

# OpenAI の text-embedding-ada-002 を使用
response = deploy_client.predict(
    endpoint="openhack-text-embedding-ada-002",
    inputs={"input": ["What is Apache Spark?"]}
)

# 結果を取得
embeddings = [e["embedding"] for e in response.data]
print(embeddings)

### Managed EmbeddingsとGTEを用いたベクター検索インデックスの作成

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/llm-rag-data-prep-3.png?raw=true" style="float: right; width: 600px; margin-left: 10px">

Managed Embeddingsを使用することで、Databricksが埋め込みを自動的に計算してくれます。これにより、Databricksの利用を簡単に始められるモードとなっています。

ベクター検索インデックスは、埋め込みを提供する**ベクター検索エンドポイント**を使用します。（これをベクター検索APIエンドポイントと考えることができます。）

複数のインデックスが同じエンドポイントを利用することも可能です。

では、インデックスを作成してみましょう。


In [0]:
from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient()

if not endpoint_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME):
    vsc.create_endpoint(name=VECTOR_SEARCH_ENDPOINT_NAME, endpoint_type="STANDARD")

wait_for_vs_endpoint_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME)
print(f"Endpoint named {VECTOR_SEARCH_ENDPOINT_NAME} is ready.")

# 10分～15分ほどかかります。

1. 現在のノートブックの左型タブにある`Workspace (Ctrl + Alt + E)`を選択し、現在のディレクトリ（`contents`)を表示
1. ケバブメニュー（`︙`）を選択し、`作成` -> `Genieスペース`を選択 *1
1. Genieスペースの作成画面にて下記セルの出力結果を設定して`Save`を選択
1. チャットウィンドウにて、`データセットに含まれるテーブルについて説明して`という質問の回答が来ることを確認

*1 Genie スペースを作成できない場合には、下記の手順を実施して Genie スペースの有効化が必要です

1. Databricks Workspace にて右上のユーザーアイコンを選択後、`Previews`を選択
1. `Genie`の有効化に関するトグルを`On`に設定
1. ページをリロード後、Databricks Workspace の左メニューにて Genie が追加されたことを確認 

参考リンク

- [AI/BI Genie スペースとは](https://learn.microsoft.com/ja-jp/azure/databricks/genie/)
- [Use trusted assets in AI/BI Genie spaces](https://learn.microsoft.com/ja-jp/azure/databricks/genie/trusted-assets)
- [効果的な Genie スペースをキュレーションする](https://learn.microsoft.com/ja-jp/azure/databricks/genie/best-practices)


<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/index_creation.gif?raw=true" width="600px" style="float: right">

ベクター検索エンドポイントは[Vector Search Endpoints UI](#/setting/clusters/vector-search)で確認できます。エンドポイント名をクリックすると、そのエンドポイントで提供されるすべてのインデックスを表示できます。

### ベクター検索インデックスの作成

ここからは、Databricksにインデックスを作成するよう依頼するだけです。

Managed Embedding Indexの場合、必要なのはテキストカラムと使用する埋め込みファウンデーションモデル（`GTE`）を指定するだけです。  
Databricksが埋め込みの計算を自動的に行います。

これはAPIを使用して行うことも、Unity Catalog Explorerメニュー内で数回のクリックで行うこともできます。


In [0]:
# Openhack ではGUIで実施する

from databricks.sdk import WorkspaceClient
import databricks.sdk.service.catalog as c

#The table we'd like to index
source_table_fullname = f"{catalog}.{db}.product_documentation"
# Where we want to store our index
vs_index_fullname = f"{catalog}.{db}.product_documentation_vs_index"

if not index_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname):
  print(f"Creating index {vs_index_fullname} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
  try:
    vsc.create_delta_sync_index(
      endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
      index_name=vs_index_fullname,
      source_table_name=source_table_fullname,
      pipeline_type="TRIGGERED",
      primary_key="id",
      embedding_source_column='content', #The column containing our text
      # embedding_model_endpoint_name='databricks-gte-large-en' #The embedding endpoint used to create the embeddings
      embedding_model_endpoint_name='openhack-text-embedding-ada-002'
    )
  except Exception as e:
    display_quota_error(e, VECTOR_SEARCH_ENDPOINT_NAME)
    raise e
  #Let's wait for the index to be ready and all our embeddings to be created and indexed
  wait_for_index_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
else:
  #Trigger a sync to update our vs content with the new data saved in the table
  wait_for_index_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
  vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname).sync()

print(f"index {vs_index_fullname} on table {source_table_fullname} is ready")

In [0]:
with open("/Volumes/trainer_catalog/default/src_data/openhack-llm/faq.html", "r", encoding="utf-8") as f:
  document_html = f.read()

displayHTML(document_html)

## 類似コンテンツの検索

これで準備は完了です。DatabricksはDelta Live Table内の新しいエントリを自動的にキャプチャし、同期します。

データセットのサイズやモデルの規模に応じて、インデックス作成が始まり、埋め込みをインデックス化するまで数秒かかる場合があります。

では、実際に試して類似コンテンツを検索してみましょう。

*補足: `similarity_search` ではフィルターパラメータもサポートされています。これはRAGシステムにセキュリティレイヤーを追加するのに便利です。例えば、ユーザーの所属部門に基づいて特定のデータをフィルタリングすることで、機密性の


In [0]:
import mlflow.deployments
deploy_client = mlflow.deployments.get_deploy_client("databricks")

question = "バッテリー交換"

results = vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname).similarity_search(
  query_text=question,
  columns=["url", "content"],
  num_results=1)
docs = results.get('result', {}).get('data_array', [])
docs